In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from pathlib import Path
from tqdm import tqdm
import pdfplumber

In [2]:
OEP = "https://www.oep.org.bo"
LISTAS = Path("listas/2019")

In [15]:
def descargar_listas():
    os.makedirs(LISTAS, exist_ok=True)
    html = BeautifulSoup(
        requests.get(f"{OEP}/elecciones-generales-2019/").text,
        "html.parser",
    )
    for entry in html.select(".et_pb_module"):
        header = entry.select("h5")
        if (
            header
            and "listas finales de candidaturas habilitadas"
            in header[0].get_text().lower()
        ):
            block = entry.select(".et_pb_toggle_content")[0]
            break
    documentos = [
        {
            "partido": link.get_text(),
            "url": f"{OEP}/{link['href']}",
        }
        for link in block.select("a")
    ]
    for documento in tqdm(documentos):
        path = LISTAS / f"{documento['partido'].replace(' ', '_')}.pdf"
        with open(path, "wb") as f:
            response = requests.get(documento["url"])
            f.write(response.content)
        documento["path"] = path
    return documentos

In [16]:
listas = descargar_listas()

100%|██████████| 9/9 [00:07<00:00,  1.25it/s]


In [17]:
listas

[{'partido': 'Comunidad Ciudadana',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2019/10/Candidaturas_Finales_CC_EG_2019.pdf',
  'path': PosixPath('listas/2019/Comunidad_Ciudadana.pdf')},
 {'partido': 'Frente Para la Victoria',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2019/10/Candidaturas_Finales_FPV_EG_2019.pdf',
  'path': PosixPath('listas/2019/Frente_Para_la_Victoria.pdf')},
 {'partido': 'Movimiento Tercer Sistema',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2019/10/Candidaturas_Finales_MTS_EG_2019.pdf',
  'path': PosixPath('listas/2019/Movimiento_Tercer_Sistema.pdf')},
 {'partido': 'Unidad Cívica Solidaridad',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2019/10/Candidaturas_Finales_UCS_EG_2019.pdf',
  'path': PosixPath('listas/2019/Unidad_Cívica_Solidaridad.pdf')},
 {'partido': 'Movimiento al Socialismo – Instrumento Político por la Soberanía de los Pueblos',
  'url': 'https://www.oep.org.bo//wp-content/uploads/2019/10/Candidaturas_Finales_MAS

In [84]:
def extraer_datos(pdf_path, partido):
    habilitados = []
    columns = [
        "nro",
        "candidatura",
        "posición",
        "nombres",
        "apellido_paterno",
        "apellido_materno",
        "genero",
        "lugar",
        "titularidad",
        "fecha_nacimiento",
        "nro_documento",
    ]
    pdf = pdfplumber.open(pdf_path)
    for page in pdf.pages:
        habilitados.extend(
            [
                row
                for row in page.extract_table({"vertical_strategy": "lines_strict"})
                if row[0] != "NRO"
            ]
        )
    df = pd.DataFrame(habilitados, columns=columns)
    df.candidatura = df.candidatura.replace(
        {
            "Presidenta -\nPresidente": "Presidente",
            "Vicepresidenta -\nVicepresidente": "Vicepresidente",
            "Diputadas(os)\nEspecial de las\nNaciones y Pueblos\nI.O.C.": "Diputados de Circunscripciones Especiales",
            "Diputadas(os)\nPlurinominales": "Diputados de Circunscripciones Plurinominales",
            "Diputadas(os)\nUninominales": "Diputados de Circunscripciones Uninominales",
            "Representantes\nante Organismos\nParlamentarios\nSupraestatales": "Representantes Supraestatales",
            "Senadoras -\nSenadores": "Senadores",
        }
    )

    df["nombre_completo"] = df[
        ["nombres", "apellido_paterno", "apellido_materno"]
    ].apply(lambda _: " ".join([i.strip().replace("\n", " ") for i in _]), axis=1)
    df.genero = df.genero.replace({"Masculino": "M", "Femenino": "F"})
    df[["departamento", "descripcion"]] = df.lugar.apply(
        lambda _: pd.Series((_.replace(" Circunscripción", "\nCircunscripción").split("\n") + [""])[:2])
    )
    df.departamento = df.departamento.replace({"Bolivia": "Nacional"})
    df.titularidad = df.titularidad.str.upper()
    df.fecha_nacimiento = pd.to_datetime(df.fecha_nacimiento, format="%Y-%m-%d")
    df.insert(0, "partido", partido)

    return df[
        [
            "partido",
            "candidatura",
            "departamento",
            "posición",
            "titularidad",
            "nombre_completo",
            "nro_documento",
            "genero",
            "fecha_nacimiento",
            "descripcion",
        ]
    ]

In [85]:
df = pd.concat([extraer_datos(lista["path"], lista["partido"]) for lista in listas])

In [86]:
df

,partido,candidatura,departamento,posición,titularidad,nombre_completo,nro_documento,genero,fecha_nacimiento,descripcion
0,Comunidad Ciudadana,Presidente,Nacional,1,TITULAR,CARLOS DIEGO DE MESA GISBERT,110761,M,1953-08-12,
1,Comunidad Ciudadana,Vicepresidente,Nacional,1,TITULAR,GUSTAVO PEDRAZA MERIDA,2957734,M,1963-07-31,
2,Comunidad Ciudadana,Diputados de Circunscripciones Especiales,Beni,1,TITULAR,MATILDE NOZA VARGAS,7604326,F,1979-03-14,
3,Comunidad Ciudadana,Diputados de Circunscripciones Plurinominales,Beni,1,TITULAR,DURBY ANDREA BLANCO BRAVO,7619514,F,1994-03-08,
4,Comunidad Ciudadana,Diputados de Circunscripciones Plurinominales,Beni,1,SUPLENTE,MARCELO ARAMAYO CHAVEZ,4198104,M,1988-11-04,
...,...,...,...,...,...,...,...,...,...,...
126,Partido de Acción Nacional Boliviano,Diputados de Circunscripciones Uninominales,Tarija,1,TITULAR,RICHARD DOMINGUEZ GARNICA,5004724,M,1978-01-21,Circunscripción
127,Partido de Acción Nacional Boliviano,Diputados de Circunscripciones Uninominales,Tarija,1,SUPLENTE,MIRIAN CRUZ ARANCIBIA,5830461,F,1976-12-23,Circunscripción
128,Partido de Acción Nacional Boliviano,Senadores,Tarija,1,TITULAR,ZULMA CORONADO CHURA,5004412,F,1979-06-02,
129,Partido de Acción Nacional Boliviano,Senadores,Tarija,1,SUPLENTE,ROY ELIO CONDORI ACEVEDO,7196408,M,1991-12-28,


In [88]:
df.to_parquet("datos/2019.parquet")